# The Task
The task is to place **products** into the **fixture**, maximizing product profit subject to the usual knapsack problem constraints:

-   **0/1 constraint:** there must be either zero or one of each product on the shelves
-   **shelf capacity constraint:** the sum of the product widths must not exceed the shelf width

`planogram.py` contains a minimal implementation.

To proceed:

1.  clone this repository
2.  **working on a new branch**, complete the implementation of `planogram.py:planogram()`.
    You are free to use your choice of additional libraries;
    make sure to update `pyproject.toml` / `requirements.txt` as necessary.
3.  get the code back to us:
    -   **private** repository on bitbucket (preferred) github, etc, or
    -   zip/tar project files and email us


In [1]:
import numpy as np
import pandas as pd
import itertools
import argparse

In [2]:
# Read in fixtures
fixture = pd.read_csv("fixture.csv")
fixture

,shelf_no,shelf_width_cm
0,1,25.4
1,2,60.0
2,3,60.0
3,4,90.0
4,5,90.0


In [3]:
# Read in Products
products = pd.read_csv("products.csv")
products.head()

,product_id,product_width_mm,profit
0,p6863,75,223.53
1,p5649,51,1183.93
2,p1592,65,214.08
3,p6878,46,81.36
4,p7897,40,182.46


In [4]:
# Check if datatypes are in order
print(products.info(), "\n")

print(fixture.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_id        99 non-null     object 
 1   product_width_mm  99 non-null     int64  
 2   profit            99 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.4+ KB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   shelf_no        5 non-null      int64  
 1   shelf_width_cm  5 non-null      float64
dtypes: float64(1), int64(1)
memory usage: 208.0 bytes
None


# Our Solution

- We'll start shelving products according to their `profit_per_mm`, as that allows us to maximise the space on our shelves to earn the most profit.


- After that we will create a function that will permutate the order of the shelves. That function will compare the profits made by each permutation and pick the the most profitable one to generate our results. 


In [7]:
# %load planogram.py

# Function to convert shelf measurement to mm
def create_shelf_mm(fixture):
    fixture["shelf_width_mm"] = fixture["shelf_width_cm"]*10
    return fixture

# Function to arrange products according to profits per milimeter
def highest_profits_first(fixture, products):
    # Create mm shelf measurement
    create_shelf_mm(fixture) 
    
    # Sort Products by Profit_per_mm
    products["profit_per_mm"] = products["profit"]/products["product_width_mm"]
    products = products.sort_values("profit_per_mm", ascending=False)
    
    # Create new ideal_shelf column
    products["ideal_shelf"]= "None"
    
    # Set Total Profit at Zero
    total_profit = 0
    
    # Loop through all shelves
    for shelf_num in range(fixture.shape[0]):
        # Reset Master ideal shelf list
        ideal_shelf_list = []
        # Reset current shelf list
        items_on_shelf = []
        
        shelf_label = fixture["shelf_no"].iloc[shelf_num]
        space_left = fixture["shelf_width_mm"].iloc[shelf_num]
        
        # Loop through all products
        for product_num in range(products.shape[0]):
            current_shelf = products["ideal_shelf"].iloc[product_num]
            product_width = products["product_width_mm"].iloc[product_num]
            product_profit = products["profit"].iloc[product_num]
            product_id = products["product_id"].iloc[product_num]
            
            # If a product isn't shelved yet and can fit on a shelf
            if space_left >= product_width and current_shelf == "None":
                # Add product onto shelf and master list
                ideal_shelf_list.append(shelf_label)
                # Calculate amount of space left
                space_left -= product_width
                # Add product profit to total profit
                total_profit += product_profit
                # Append item onto a list
                items_on_shelf.append(product_id)
            else: 
                # If item doesn't fit, append its current status shelf onto master list
                ideal_shelf_list.append(current_shelf)
        # Update ideal_shelf column
        products["ideal_shelf"]=ideal_shelf_list
    
    return total_profit, products


# Final function to create a dataframe with most profitable placement of products
def planogram(fixture, products):
    """
    Arguments:
    - fixture :: DataFrame[["shelf_no", "shelf_width_cm"]]
    - products :: DataFrame[["product_id", "product_width_mm", "profit"]]

    Returns: DataFrame[["shelf_no", "product_id"]]
    """
    # Create a list of all possible permutations for index
    original_index = fixture.index.tolist()
    fixture_permutate = [list(x) for x in list(set(itertools.permutations(original_index)))]
    # Create empty list to nest profits and index
    profit_list = []
    #print (fixture_permutate)
    for index in fixture_permutate:
        profit = highest_profits_first(fixture.iloc[index], products)[0]
        df = highest_profits_first(fixture.iloc[index], products)[1]
        # Append a tuple of profit, index
        profit_list.append((profit, index, df))
    
    products = max(profit_list)[2]
    best_option_profit = max(profit_list)[0]
    max_possible_profit = products["profit"].sum()

    # Create a new column called "shelf_no"
    products["shelf_no"] = products["ideal_shelf"].copy()
    # Create a new df that does not half items not on shelves
    df = products[products["shelf_no"]!="None"][["shelf_no", "product_id"]]
    return df.sort_values("shelf_no")



if __name__ == "__main__":
    import numpy as np
    import pandas as pd
    import itertools

    import argparse

    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--products",
        type=argparse.FileType(mode="r"),
        default="products.csv",
        help="products input file",
    )
    parser.add_argument(
        "--fixture",
        type=argparse.FileType(mode="r"),
        default="fixture.csv",
        help="fixture input file",
    )
    parser.add_argument(
        "--out", "-o", default="solution.csv", help="solution output file"
    )
    
    ### Dummy argument added here that allows function to run in Jupyter Notebooks
    ### Delete next three lines if this causes an error 
    parser.add_argument(
        '-f'
    )
    ###

    args = parser.parse_args()

    fixture = pd.read_csv(args.fixture)
    products = pd.read_csv(args.products)

    solution = planogram(fixture, products)

    solution.to_csv(args.out, index=False)

    print(
        "stats:",
        solution[["shelf_no", "product_id"]]
        .merge(fixture, on="shelf_no")
        .merge(products, on="product_id")
        .assign(n_products=1)
        .pivot_table(
            index=["shelf_no", "shelf_width_cm"],
            values=["n_products", "profit", "product_width_mm"],
            aggfunc=np.sum,
            margins=True,
        ),
        sep="\n",
    )



/Users/samuelhe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


stats:
                         n_products  product_width_mm    profit
shelf_no shelf_width_cm                                        
1        25.4                     5               253    520.50
2        60.0                    11               600   1579.29
3        60.0                    13               598   8719.42
4        90.0                    17               897   3201.25
5        90.0                    17               890   5103.60
All                              63              3238  19124.06


# Evaluation

In [8]:
def check_final_results(fixture, products):
    """
    Arguments:
    - fixture :: DataFrame[["shelf_no", "shelf_width_cm"]]
    - products :: DataFrame[["product_id", "product_width_mm", "profit"]]

    Returns: DataFrame[["shelf_no", "product_id"]]
    """
    # Create a list of all possible permutations for index
    original_index = fixture.index.tolist()
    fixture_permutate = [list(x) for x in list(set(itertools.permutations(original_index)))]
    # Create empty list to nest profits and index
    profit_list = []
    #print (fixture_permutate)
    for index in fixture_permutate:
        profit = highest_profits_first(fixture.iloc[index], products)[0]
        df = highest_profits_first(fixture.iloc[index], products)[1]
        # Append a tuple of profit, index
        profit_list.append((profit, index, df))
    
    products = max(profit_list)[2]
    best_option_profit = max(profit_list)[0]
    max_possible_profit = products["profit"].sum()
    
    return products

In [10]:
# Create df to check results
results_check = check_final_results(fixture, products)

/Users/samuelhe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [17]:
# Create
pd.set_option('display.max_rows', 99)
results_check.sort_values("profit_per_mm", ascending=False)

,product_id,product_width_mm,profit,profit_per_mm,ideal_shelf
90,p4833,54,1408.21,26.077963,3
8,p7122,65,1668.14,25.663692,3
1,p5649,51,1183.93,23.214314,3
53,p9129,52,862.89,16.594038,3
79,p8909,29,450.83,15.545862,3
96,p3438,56,684.51,12.223393,3
13,p6086,66,706.19,10.699848,3
26,p2722,43,421.59,9.804419,3
47,p5083,48,393.15,8.190625,3
66,p9134,41,329.42,8.034634,3


In [21]:
# Maximum possible profit
results_check.profit.sum()

21732.75

### Analysis on the effectiveness of our solution. 
![](./assets/profit.png)

- **Profitability** - Our shelves contain products with a total profit of 19124.06, out of a maximum possible profit of 21732.75.


- **Free space left** - Based on the items sitting on the shelf in our solution, we have a collective 16mm of free space left. 


- **The ones that got away** -  Our function has maximised `profit_per_mm`, except for the four items shown above, which has higher `profit_per_mm` compared to the final item that made it onto a shelf, p4773. As we have only 16mm of space left, we can only reasonably consider adding two of these four items onto the shelf to increase profitability.


- **Future improvements** - Although, we'd like to believe that we've come up with the best possible scenario, the fact that we have left empty space on the shelves means that there just might be better ways of maximising profitability!
